In [2]:
import os
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import col, length, regexp_replace

def spark_read_csv_from_os(spark, file_path, schema, header=True, **options):
    """
    Reads a CSV file from the operating system into a Spark DataFrame.

    Args:
        spark: The SparkSession object.
        file_path: The path to the CSV file.  Can be a local path or a path
                   that your Spark environment can access (e.g., if you're
                   using a distributed file system like HDFS).
        header (bool, optional): Whether the CSV file has a header row. Defaults to True.
        inferSchema (bool, optional): Whether to infer the schema from the data. Defaults to True.
        **options: Additional options to pass to the Spark CSV reader.  See
                   the Spark documentation for available options like `delimiter`,
                   `quote`, `escape`, etc.

    Returns:
        A Spark DataFrame representing the CSV data, or None if there's an error.

    Raises:
       FileNotFoundError: If the file path doesn't exist.
    """
    try:
        df = spark.read.csv(file_path, header=header, inferSchema=False, schema=schema, **options)
        return df
    except FileNotFoundError:
        print(f"Error: File not found at path: {file_path}")
        return None
    except Exception as e:  # Catch other potential exceptions (e.g., parsing errors)
        print(f"An error occurred while reading the CSV: {e}")
        return None
    
def pandas_read_csv(file_path,**options):
    """
        Read small volume of data only using read.csv
        Args:
            **Options ----> Any
    """
    try:
        df = pd.read_csv(file_path,**options)
        return df
    except FileNotFoundError:
        print(f"Error: File not found at path: {file_path}")
        return None
    except Exception as e:  # Catch other potential exceptions (e.g., parsing errors)
        print(f"An error occurred while reading the CSV: {e}")
        return None
    
def construct_sql_schema(**kwargs):
    """
        Args: kwargs path and sep -->>> Any
        this function is best practice to compute large amount of data to not reading schema metadata
        recommendation : 
    """

    fields = []
    type_mapping = {
        "varchar": StringType(),
        "nvarchar": StringType(),
        "int": IntegerType(),
        "bigint": LongType(),
        "date": DateType(),
        "decimal": DecimalType
    }

    df = pandas_read_csv(kwargs["path"],sep=kwargs["sep"])

    for row in df.itertuples():
        try:
            name, data_type_str = row.DataType.split("(", 1) if "(" in row.DataType else (row.DataType,"")
            name = name.strip()
            data_type_str = data_type_str[:-1].strip()
            parts = data_type_str.split(",")
            name_lower = name.lower()

            for keyword,spark_type in type_mapping.items():
                if keyword in name_lower:
                    if spark_type == DecimalType:
                        data_type = DecimalType() if not data_type_str else DecimalType(int(parts[0]),int(parts[1]))
                        fields.append(StructField(row.ColumnName, data_type, True))
                    else:
                        data_type = spark_type
                        fields.append(StructField(row.ColumnName, data_type, True))
                    break
        except Exception as e:  # Catch other potential errors
            print(f"Error processing file in construct schema {kwargs["path"]}: {e}")
            return None
    return StructType(fields)

def validateDecimal(**kwargs):
    df_contents = kwargs["df_contents"]
    is_valid = False
    errors = []
    for field in kwargs["dtypes"]:
        colName = field.name
        dType = str(field.dataType)
        if "decimal" in dType.lower():
            #print(colName)
            df_cleaned = df_contents.withColumn(
                f"{colName}_cleaned",
                regexp_replace(col(colName), "[^0-9.]", "")
            )
            df_empty = df_cleaned.filter((col(f"{colName}_cleaned")).isNull()) # is null due to schema defined as decimal if we use inferSchema its a string
            empty_count = df_empty.count()

            if empty_count > 0:
                is_valid = True
                sample_size = min(100, empty_count)
                invalid_rows = df_empty.select(colName).take(sample_size)
                error_msg = (f"Invalid {colName} values (containing only non-numeric characters) in {FullPathSchema}. Total count: {empty_count}")
                errors.append(error_msg)
            df_contents = df_contents.drop(f"{colName}_cleaned") 
    msg = "\n".join(errors) if errors else "Data Quality Checks Passed."
    return is_valid, msg, df_contents

if __name__ == "__main__":
    path = "/mnt/apps/Files/Config/master_job.csv"
    pathSchema = "/mnt/apps/Files/Schema/"
    outputFile = "/mnt/apps/Files/data-movement/"

    df = pandas_read_csv(path,sep="|")
    df = df.query("JobName == 'PAT'")

    for row in df.itertuples():  # Collects all data to the driver - NOT recommended for large datasets
        filePath = row.SourceDirectory + '/' + row.FileName + '*' + '.' + row.FileType
        filePath = filePath.replace("/gcs", "/Files")
        dataMovement = outputFile + row.JobName + '/'
        print(dataMovement)
        FullPathSchema = pathSchema + row.FileName + '.' + row.FileType
        #spark.stop()
        spark = SparkSession. \
                builder. \
                appName(f"{row.JobName}").getOrCreate()
        df_dtype = construct_sql_schema(path=FullPathSchema, sep="|")
        df = spark_read_csv_from_os(spark, filePath, schema=df_dtype, quote='"', sep=row.Delimiter)
        result, dqc_msg, df_final = validateDecimal(dtypes=df_dtype, df_contents=df)
        print(df.rdd.getNumPartitions()) # rdd create while read csv
        #df_count = df_final.count()
        #print(df_count)
        df.limit(10)
        num_cores = spark.sparkContext.defaultParallelism
        #print(f"Type of num_cores: {type(num_cores)}") # Add this line!
        num_partitions = num_cores * 1
        print(f"Number of partitions: {num_partitions}")
        partitioned_df = df.repartition(num_partitions)
        if result:
            df.write.format("csv"). \
                    mode("overwrite"). \
                    option("header", "true"). \
                    save(dataMovement)
            print(dqc_msg)
        else:
            df.write.csv(
            path=dataMovement,
            mode="overwrite",  # Overwrite existing files
            header=True,       # Include header row (optional)
            sep=row.Delimiter
            # coalesce = True # DO NOT USE COALESCE for large datasets unless you *really* need one file.
            )
            print(dqc_msg)
    spark.stop()

/mnt/apps/Files/data-movement/PAT/


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/02/17 02:46:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


1
Number of partitions: 16
Invalid AMT_RPT values (containing only non-numeric characters) in /mnt/apps/Files/Schema/etl4pat.csv. Total count: 1
Invalid AMT_ORG values (containing only non-numeric characters) in /mnt/apps/Files/Schema/etl4pat.csv. Total count: 2


In [9]:
import os
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import input_file_name

#outputFile = "/mnt/apps/Files/ETL4/LAS/customers.csv"
outputFile = "/mnt/apps/Files/data-movement/Renova/part-000*"
FullPathSchema = "/mnt/apps/Files/Schema/customers.csv"
ParquetPath = "/mnt/apps/Files/data-movement/Parquet/Renova/part-000*"

df_dtype = construct_sql_schema(path=FullPathSchema, sep="|")
print(df_dtype)

spark = SparkSession.builder.appName("TestReadCSV").getOrCreate()

#CSV reader
# df = spark.read.csv(outputFile, sep="|", header=True, schema=df_dtype, inferSchema=False)

# df_with_filename = df.withColumn("filename", input_file_name())
# null_count = df_with_filename.filter(col("Index").isNotNull())
# df_count = null_count.count()
# print(df_count)
# #null_count.limit(10).show(truncate=False)
# null_count.createOrReplaceTempView("my_parquet_table")
# #df_parquet_count = null_count_parquet.count()
# #print(df_parquet_count)
# null_count = spark.sql(
#     """
#     SELECT country, cnt, sum(cnt) over () total_all
#     FROM (
#     SELECT Country, COUNT(1) AS cnt
#         FROM my_parquet_table
#         GROUP BY Country
#     ) Z
# """)
# print(null_count.count())
# null_count.show(n=5, truncate=False) #default 20

#parquet part
df_read_parquet = spark.read.parquet(ParquetPath, schema=df_dtype)
null_count_parquet = df_read_parquet.filter(col("Index").isNotNull())
null_count_parquet.createOrReplaceTempView("my_parquet_table")
#df_parquet_count = null_count_parquet.count()
#print(df_parquet_count)
null_count_parquet = spark.sql(
    """
    SELECT country, cnt, sum(cnt) over () total_all
    FROM (
    SELECT Country, COUNT(1) AS cnt
        FROM my_parquet_table
        GROUP BY Country
    ) Z
""")
print(null_count_parquet.count())
null_count_parquet.show(n=5, truncate=False) #default 20

spark.stop()


StructType([StructField('Index', IntegerType(), True), StructField('Customer Id', StringType(), True), StructField('First Name', StringType(), True), StructField('Last Name', StringType(), True), StructField('Company', StringType(), True), StructField('City', StringType(), True), StructField('Country', StringType(), True), StructField('Phone 1', StringType(), True), StructField('Phone 2', StringType(), True), StructField('Email', StringType(), True), StructField('Subscription Date', DateType(), True), StructField('Website', StringType(), True)])


25/02/17 03:41:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/17 03:41:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/17 03:41:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


243


25/02/17 03:41:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/17 03:41:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/17 03:41:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/02/17 03:41:29 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---------------------------------+------+---------+
|country                          |cnt   |total_all|
+---------------------------------+------+---------+
|Chad                             |284690|70000000 |
|Paraguay                         |284550|70000000 |
|Anguilla                         |284480|70000000 |
|Macao                            |288645|70000000 |
|Heard Island and McDonald Islands|282870|70000000 |
+---------------------------------+------+---------+
only showing top 5 rows



In [ ]:
path = "/mnt/apps/Files/Config/master_job.csv"
pathSchema = "/mnt/apps/Files/Schema/"

df = pandas_read_csv(path,sep="|")
df = df.query("JobName == 'PAT'")

for row in df.itertuples():  # Collects all data to the driver - NOT recommended for large datasets
    filePath = row.SourceDirectory + '/' + row.FileName + '.' + row.FileType
    FullPathSchema = pathSchema + row.FileName + '.' + row.FileType
    spark = SparkSession.builder.appName(f"{row.JobName}").getOrCreate()
    df_dtype = construct_sql_schema(path=FullPathSchema, sep="|")
    df = spark_read_csv_from_os(spark, filePath, schema=df_dtype, quote='"', sep="|")
    # Handle the error, e.g., skip the file, log the error, etc.
    df.show()
"""     result, dqc_msg, df_final = validateDecimal(dtypes=df_dtype, df_contents=df)
    df_final.show()
    if result:
        print(dqc_msg)
    else:
        print(dqc_msg) """
spark.stop()


In [ ]:
path = "/mnt/apps/Files/Config/master_job.csv"

df = pandas_read_csv(path,sep="|")
df = df.query("Flag == 1")
df.head()


In [1]:
from pyspark.sql.types import *

path = "/mnt/apps/Files/Schema/etl4pat.csv"
fields = []
type_mapping = {
    "varchar": VarcharType,
    "nvarchar": VarcharType,
    "int": IntegerType(),
    "bigint": LongType(),
    "date": DateType(),
    "decimal": DecimalType
}

df = pandas_read_csv(path,sep="|")
print(df)

for row in df.itertuples():
    name, data_type_str = row.DataType.split("(", 1) if "(" in row.DataType else (row.DataType,"")
    name = name.strip()
    data_type_str = data_type_str[:-1].strip()
    parts = data_type_str.split(",")
    name_lower = name.lower()
    print(data_type_str)

    for keyword,spark_type in type_mapping.items():
        if keyword in name_lower:
            if spark_type == VarcharType:
                data_type = VarcharType(4000) if data_type_str == "MAX" else VarcharType(int(data_type_str))
                fields.append(StructField(row.ColumnName, data_type, True))
            elif spark_type == DecimalType:
                data_type = DecimalType() if not data_type_str else DecimalType(int(parts[0]),int(parts[1]))
                fields.append(StructField(row.ColumnName, data_type, True))
            else:
                data_type = spark_type
                fields.append(StructField(row.ColumnName, data_type, True))
            break

print(fields)


NameError: name 'pandas_read_csv' is not defined

In [188]:
data_type_str = "Decimal(2,0)"
data_type, *args = data_type_str.split(")")

print(f"data_type: {data_type}")  # Output: data_type: Decimal(2,0
print(f"args: {args}")          # Output: args: ['']

data_type_str = "VARCHAR"
data_type, *args = data_type_str.split(")")

print(f"data_type: {data_type}")  # Output: data_type: VARCHAR
print(f"args: {args}")          # Output: args: []

data_type: Decimal(2,0
args: ['']
data_type: VARCHAR
args: []


In [374]:
data = "A,2"

print(len(data))

3


In [2]:
path = "/mnt/apps/Files/Schema/etl4pat.csv"
test = construct_sql_schema(path=path,sep="|")
print(test)

Error: File not found at path: /mnt/apps/Files/Schema/etl4pat.csv


AttributeError: 'NoneType' object has no attribute 'itertuples'

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *  # Import data types for clarity

# Create a SparkSession (if you don't have one already)
spark = SparkSession.builder.appName("DataTypeExample").getOrCreate()

# Sample data (replace with your actual data)
data = [("Alice", 25, 2000.00), ("Bob", 30, 2000.00), ("Charlie", 22, 2000.00)]

# Define the schema explicitly (best practice)
schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("height", DecimalType(2,0), True)
])

df = spark.createDataFrame(data, schema=schema)

# Display the schema
df.printSchema()

# Stop the SparkSession (good practice)
spark.stop()

In [ ]:
data = "/mnt/apps/gcs/ETL4/CONFIG/etl4pat*.csv"



In [ ]:
from pyspark.sql import SparkSession

def writeOptions(df, path, **kwargs):
    base_options = {  # Keep this separate
        "header": "true",
        "delimiter": "|",
        "quote": '"',
        "mode":"overwrite",
        "format":"csv"
    }

    # Correct way to merge options:
    all_options = base_options.copy()  # Create a copy to avoid modifying base_options
    all_options.update(kwargs)       # Add or overwrite kwargs
    print(all_options)

    df.write.options(**all_options).save(path)

# Example usage (important: complete example):
spark = SparkSession.builder.appName("Example").getOrCreate()
data = [("Alice", 25), ("Bob", 30)]
df = spark.createDataFrame(data, ["name", "age"])

dataMovement = "path/to/save.csv" # Or your actual path

writeOptions(df, dataMovement)  # Now works correctly

spark.stop()  # Don't forget to stop the SparkSession

{'header': 'true', 'delimiter': '|', 'quote': '"', 'mode': 'overwrite', 'format': 'csv'}
